https://huggingface.co/learn/nlp-course/chapter3/4?fw=pt

https://www.kaggle.com/code/prakharrathi25/sentiment-analysis-using-bert

In [1]:
#!pip install transformers[torch]
#!pip install accelerate -U

In [ ]:
access_token = ""

In [3]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, default_data_collator, EarlyStoppingCallback, IntervalStrategy
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score # Import all needed metrics
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
dataset = pd.read_csv('/content/drive/MyDrive/H.IAAC/datasets/filtered_output_fT.csv')

In [5]:
dataset

,level_0,index,Unnamed: 0,reviewId,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,sentiment,lengthContent,Language
0,9031,13589,13589,c9245f53-f717-4d6a-a89b-e370640574cd,Every very good thing is expensive,3,0,26.10.4.100,2023-09-14 02:17:03,NaN,NaN,26.10.4.100,1,34,en
1,6761,9951,9951,4180b110-d672-492c-9551-c8aaf60b1d5d,"The last few days, the app keeps crashing . Ve...",1,1,26.21.0.100,2023-11-06 03:45:52,NaN,NaN,26.21.0.100,0,100,en
2,4844,7070,7070,dac3ff58-e9e4-4995-93c1-a54140d6ae91,Irrelevant .. I don't like having apps that I ...,1,0,22.2.0.100,2023-11-18 17:07:42,NaN,NaN,22.2.0.100,0,87,en
3,16081,22809,22809,0a03a490-bef5-403e-a5e6-25930f9f7e41,Every time i go to (my orders) its always load...,3,0,24.12.6.100,2022-07-28 11:53:04,NaN,NaN,24.12.6.100,1,89,en
4,7326,10674,10674,81dcbdcd-5de7-4c1b-ad31-aa97a8237205,Great experience with company also fast return...,5,0,26.21.0.100,2023-11-03 22:40:37,NaN,NaN,26.21.0.100,2,68,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9797,14565,14565,2486626f-e174-40f4-a1bb-69dc1ed5863e,"Ok this app it great, your packages come to yo...",2,0,NaN,2023-08-04 01:11:11,NaN,NaN,NaN,0,485,en
9996,5363,7957,7957,71b91784-824a-43d1-9304-5566aafcb0c4,Discontinued Tab app but can not install Amazo...,1,2,26.21.0.100,2023-11-13 08:16:43,NaN,NaN,26.21.0.100,0,295,en
9997,45,83,83,c5376035-f0c5-430f-a4db-dafc8cda6ce6,worst experience with AMAZON.Even Prime member...,1,0,26.23.4.100,2024-01-08 09:09:42,NaN,NaN,26.23.4.100,0,113,en
9998,6113,9113,9113,96a248a6-645e-487d-ba71-462d20aed048,I don't know what you did in your last update ...,1,3,26.21.0.100,2023-11-07 00:50:21,NaN,NaN,26.21.0.100,0,367,en


In [6]:
dataset.shape

(10000, 15)

In [7]:
sentiment_counts_balanced = dataset['sentiment'].value_counts().sort_index()
print(sentiment_counts_balanced)

0    4758
1    2322
2    2920
Name: sentiment, dtype: int64


In [8]:
# Load the pre-trained BERT model and tokenizer
#mod = "mistralai/Mistral-7B-v0.1"
model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3, token=access_token)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, tokenizer, dataframe, max_length=512):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]['content']
        label = self.data.iloc[idx]['sentiment']
        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [10]:
# Create train and validation datasets
comments = dataset[['content', 'sentiment']]
train_data, val_data = train_test_split(comments, test_size=0.2, random_state=42)

In [11]:
# Define batch size and create data loaders
batch_size = 8  # Adjust as needed
train_dataset = CustomDataset(tokenizer, train_data)
val_dataset = CustomDataset(tokenizer, val_data)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=default_data_collator)
val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=default_data_collator)

In [12]:
# Define training arguments with early stopping
training_args = TrainingArguments(
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    output_dir="./bert_sentiment_model",
    evaluation_strategy="steps",
    eval_steps=100,  # Adjust as needed
    save_total_limit=2,  # Adjust as needed. Only last 2 models are saved. Older ones are deleted.
    num_train_epochs=3,  # Adjust as needed
    save_steps=500,  # Adjust as needed
    logging_dir="./logs",
    metric_for_best_model="eval_loss",  # Use validation loss for early stopping
    load_best_model_at_end=True
)

In [13]:
def compute_metrics(p):
  pred, labels = p
  pred = np.argmax(pred, axis=1)
  accuracy = accuracy_score(y_true=labels, y_pred=pred)
  recall = recall_score(y_true=labels, y_pred=pred, average='macro')
  precision = precision_score(y_true=labels, y_pred=pred, average='macro')
  f1 = f1_score(y_true=labels, y_pred=pred, average='macro')  # Add line for F1 score
  return  {
            'accuracy': accuracy,
            'recall': recall,
            'precision': precision,
            'f1': f1  # Include F1 score in the dictionary
          }

In [14]:
# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=default_data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

In [15]:
# Fine-tune the model
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
100,No log,0.783587,0.674000,0.578414,0.608430,0.524676
200,No log,0.751400,0.679000,0.598406,0.625789,0.578268
300,No log,0.786288,0.665000,0.580298,0.571997,0.513730
400,No log,0.766089,0.639000,0.600323,0.601048,0.600599
500,0.822800,0.750933,0.669500,0.587755,0.630024,0.586778
600,0.822800,0.728850,0.674000,0.616353,0.612321,0.608921
700,0.822800,0.736737,0.686000,0.607122,0.641209,0.603419
800,0.822800,0.744785,0.676500,0.616090,0.633365,0.618235
900,0.822800,0.726642,0.670000,0.629167,0.655441,0.638110
1000,0.755100,0.721686,0.681500,0.594134,0.620399,0.575587


TrainOutput(global_step=1300, training_loss=0.7458011451134315, metrics={'train_runtime': 1752.0371, 'train_samples_per_second': 13.698, 'train_steps_per_second': 1.712, 'total_flos': 2736379544371200.0, 'train_loss': 0.7458011451134315, 'epoch': 1.3})

In [16]:
# Evaluate the model
results = trainer.evaluate()

In [17]:
results

{'eval_loss': 0.7216863632202148,
 'eval_accuracy': 0.6815,
 'eval_recall': 0.594134060743161,
 'eval_precision': 0.620399052837043,
 'eval_f1': 0.5755868507956323,
 'eval_runtime': 63.2052,
 'eval_samples_per_second': 31.643,
 'eval_steps_per_second': 3.955,
 'epoch': 1.3}

In [18]:
# Save the model
model.save_pretrained("./bert_sentiment_model")

In [19]:
path = '/content/drive/MyDrive/H.IAAC/datasets/'
output_file_path = 'bert_sentiment_model'
destination = path+output_file_path

# Save the model
model.save_pretrained(destination)

In [20]:
# You can also load the model later using:
# model = AutoModelForSequenceClassification.from_pretrained("./bert_sentiment_model")

In [21]:
# Make sure the model and input data are on the same device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,